In [ ]:
import pandas as pd
import cudf
import numpy as np
import random
from cuml.model_selection import train_test_split, GridSearchCV
from cuml.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Leer el archivo CSV
loan_data_knn = pd.read_csv('loan_data_knn_ML_python.csv')

# Establecer la semilla de aleatoriedad
np.random.seed(12345)
random.seed(12345)

In [ ]:
# Convertir 'repay_fail' a formato categoría
loan_data_knn['repay_fail'] = loan_data_knn['repay_fail'].astype('category')

# Verificar la clase de 'repay_fail'
print(loan_data_knn['repay_fail'].dtype)

# Verificar los niveles del factor
print(loan_data_knn['repay_fail'].value_counts())

In [ ]:
# Splitting inicial de los datos con muestreo estratificado
X = loan_data_knn.drop(columns='repay_fail')
y = loan_data_knn['repay_fail']

# Convertir DataFrames de pandas a cuDF para utilizar la GPU
X_cudf = cudf.DataFrame.from_pandas(X)
y_cudf = cudf.Series(y.cat.codes.astype('int32'))  # Convertir a códigos de categoría para cuDF

X_train_cudf, X_test_cudf, y_train_cudf, y_test_cudf = train_test_split(X_cudf, y_cudf, test_size=0.3, stratify=y_cudf, random_state=123)

# Verificar proporciones
print("Proporciones en el conjunto de entrenamiento:")
print(y_train_cudf.value_counts(normalize=True).to_pandas())

print("Proporciones en el conjunto de prueba:")
print(y_test_cudf.value_counts(normalize=True).to_pandas())

In [ ]:
# Verificar tamaños de los conjuntos de entrenamiento y prueba
print("Tamaño del conjunto de entrenamiento:", len(X_train_cudf))
print("Tamaño del conjunto de prueba:", len(X_test_cudf))

# Verificar proporciones de los tamaños
total_samples = len(X_cudf)
train_proportion = len(X_train_cudf) / total_samples
test_proportion = len(X_test_cudf) / total_samples
print("Proporción de entrenamiento:", train_proportion)
print("Proporción de prueba:", test_proportion)

In [ ]:
# Convertir los datos a NumPy (debido a que GridSearchCV de scikit-learn no soporta CuPy directamente)
X_train_numpy = X_train_cudf.to_numpy()
X_test_numpy = X_test_cudf.to_numpy()
y_train_numpy = y_train_cudf.to_numpy()
y_test_numpy = y_test_cudf.to_numpy()

In [ ]:
# Definir el grid de hiperparámetros
hyper_grid = {
    'n_neighbors': list(range(3, 38, 2)),
    'metric': ['minkowski'],
    'p': [1, 2, 3]  # Añadir diferentes valores de p para minkowski
}

# Definir el clasificador KNN
knn = KNeighborsClassifier()

In [ ]:
# Definir GridSearchCV
grid_search = GridSearchCV(estimator=knn, param_grid=hyper_grid, cv=5, scoring='accuracy')

# Ajustar el modelo
grid_search.fit(X_train_numpy, y_train_numpy)

# Mejor modelo
best_model = grid_search.best_estimator_

# Imprimir el mejor modelo
print("Best Model:", best_model)

# Resumen de los resultados de la búsqueda en grilla
results = pd.DataFrame(grid_search.cv_results_)

# Mostrar los resultados de la búsqueda en grilla
results = results[['param_n_neighbors', 'param_metric', 'param_p', 'mean_test_score', 'std_test_score']]
results.columns = ['#Neighbors', 'Metric', 'p', 'Mean Accuracy', 'Std Accuracy']
print(results)

# Graficar resultados en un solo gráfico con diferentes colores para cada valor de p
plt.figure(figsize=(10, 6))
ps = results['p'].unique()
colors = sns.color_palette("husl", len(ps))  # Obtener una paleta de colores

for p, color in zip(ps, colors):
    subset = results[results['p'] == p]
    plt.plot(subset['#Neighbors'], subset['Mean Accuracy'], marker='o', label=f'p = {p}', color=color)

plt.title('Resultados de la Validación Cruzada')
plt.xlabel('Número de Vecinos')
plt.ylabel('Precisión')
plt.legend(title='Minkowski p')
plt.grid(True)
plt.show()

In [ ]:
# Evaluar el mejor modelo
y_pred = best_model.predict(X_test_numpy)

# Calcular la matriz de confusión y precisión
conf_matrix = confusion_matrix(y_test_numpy, y_pred)
accuracy = accuracy_score(y_test_numpy, y_pred)

print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)

In [ ]:
# Curva ROC y AUC
y_pred_prob = best_model.predict_proba(X_test_numpy)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_numpy, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()